Basic example of particle swarm to optimize a one-variable function.

In [10]:
import pandas as pd
import networkx as nx
import random

In [120]:
# Generate the network topology - in this case, von Neumann (i.e. every node has exactly 4 neighbors)
pop_size = 40
num_neighbors = 4
graph = nx.random_regular_graph(num_neighbors, pop_size)

In [106]:
#Set the sociality (how much the node cares about the group opinion), memory(how much it cares about its personal best),
#and stability (the tendency to stay where it is)
sociality = 0.5
memory = 0.3
stability = 1 - sociality - memory

In [107]:
def f(x):
    return x**2

In [121]:
# Set initial population - random numbers btween -100 and 100
initial_values = [200*random.random() - 100 for _ in range(pop_size)]

In [122]:
Population = pd.DataFrame({'node' : list(range(40)), 'current_value' : initial_values, 'personal_best' : initial_values})

In [123]:
Population['fitness'] = Population.current_value.apply(f)

In [124]:
Population['neighbors'] = Population.node.apply(lambda x: [x] + [_ for _ in graph[x]])

In [125]:
# Find the group best - this is ugly and maybe could be cleaned up to not look so bad
Population['group_best'] = Population.node.apply(lambda x:
Population.loc[(Population.node.isin(Population.loc[x].neighbors)) &  
               (Population.fitness == Population.loc[Population.node.isin(Population.loc[x].neighbors)].fitness.min())].values[0][1])
Population['group_best_fitness'] = Population.group_best.apply(f)
Population['personal_best_fitness'] = Population.personal_best.apply(f)

In [128]:
print(Population.fitness.min())

42.79853983066854


In [129]:
def mutation(current_value):
    x = random.random()
    if x < .05: return 200*random.random() - 100
    return current_value

In [ ]:
num_iterations = 1000
for i in range(num_iterations):
    #Apply crossover
    Population.current_value = Population.apply(lambda row: stability*row.current_value + memory*row.personal_best + sociality*row.group_best, axis = 1)
    Population.current_value = Population.current_value.apply(mutation)
    Population.fitness = Population.current_value.apply(f)
    Population.personal_best = Population.apply(lambda row: row.current_value if row.fitness < row.personal_best_fitness else row.personal_best, axis = 1)
    Population['personal_best_fitness'] = Population.personal_best.apply(f)
    # Find the group best - this is ugly and maybe could be cleaned up to not look so bad
    Population['group_best'] = Population.node.apply(lambda x:
    Population.loc[(Population.node.isin(Population.loc[x].neighbors)) &  
               (Population.personal_best_fitness == Population.loc[Population.node.isin(Population.loc[x].neighbors)].personal_best_fitness.min())].values[0][2])
    Population['group_best_fitness'] = Population.group_best.apply(f)

    print(Population.fitness.min(), Population.fitness.mean())